In [1]:
import pandas as pd
import json
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import re
from dateutil import parser
import json
import ipaddress

In [2]:
ignoreList = ['.length ']
matchPatterns = []
def tokenize_filter(filter_dict):
    tokens = []
    if filter_dict is not None:
        filter_str = json.dumps(filter_dict)
        tokens = re.findall(r"\[\w']+|\[.,!?;\]", filter_str)
    return tokens

def replace_alpha_words(text):
    if text is None:
        return("")
    else:
        text = re.sub(r"\^[(\w\d|)\\+]+", 'name', text) 
        text = re.sub(r"\w+\{\}", 'name', text)
        text = re.sub(r"\^\w+",'name',text)
        text = re.sub(r"\&\w+",'name',text)
        text = re.sub(r"&ne",'name',text)
        text = re.sub(r"test\{\}",'name',text)
        text = re.sub(r"admin",'name',text)
        text = re.sub(r"benign",'name',text)
        text = re.sub(r"malicious",'name',text)
        text = re.sub(r"\$username",'name',text)
        text = re.sub(r"\$password",'name',text)
        text = re.sub(r"'[\w ]+'", '\'name\'', text)
        text = re.sub(r"\{\}\|name", 'name', text)
        text = re.sub(r"\.\*\{\}\.\*", 'name', text)
        text = re.sub(r"\{\}\.\*name\.\*", 'name', text)  
        text = re.sub(r"\^\{\}\$", 'name', text)
        text = re.sub(r"\^\{\}", 'name', text)
        text = re.sub(r"\^name\$", 'name', text)
        text = re.sub(r"test\'name\'\{\}", 'name', text)
        text = re.sub(r'this\.name',"this.",text)
        text = text.replace("test'name'{}","name")
        text = re.sub(r'name',"",text)
        text = re.sub(r'name',"",text)       
        return text
    
def replace_nums(text):
    if text is None:
        return("")
    else:
        regex = r"\d+"
        return re.sub(r"\d+", '\'name\'', text)

def replace_this_var(text):
    matched = (re.findall(r"\.[\w]+",text))
    #matchPatterns.append(matched)
    for matches in matched:
        if str(matches) != '.length':
            text = re.sub(matches,".name",text)
        else:
            matchPatterns.append(matches)
    return text

def replace_this_var_condi(text):
    regex = r"^(?!\$)\w+"
    return int(re.sub(r"\.[\w]+[\.]",".name.",text))

def comparisonOperator(text):
    pattern = r'\$(gt|eq|gte|in|lt|lte|ne|nin)'
    return 1 if re.search(pattern, text) else 0

In [3]:
labelData = pd.read_json("../No-SQL_Gen/No-SqlDataset.json")
labelData['text'] = labelData['text'].apply(lambda x: x.replace('"', "'") if isinstance(x, str) else x)
labelData.head()

,text,label
0,"{'user': '{}', 'password': '{}'}",0
1,{'username': {'$regex': '^{}'}},1
2,{'username': {'$ne': '{}'}},1
3,{'username': {'$gt': '{}'}},1
4,"{'user': 'hacker', 'password': {'$ne': ''}}",1


In [4]:
logData = pd.read_json('../Dataset/queryLogs.json')
logData.head()
logData['t'] = logData['t'].apply(lambda x: parser.parse(x['$date']).timestamp())
attr_cols = ['type', 'ns', 'command', 'planSummary', 'planningTimeMicros', 'keysExamined', 'docsExamined', 'nBatches', 'cursorExhausted', 'numYields', 'nreturned', 'queryFramework', 'reslen', 'locks', 'storage', 'cpuNanos', 'remote', 'protocol', 'durationMillis']
for col in attr_cols:
    logData[col] = logData['attr'].apply(lambda x: x.get(col, None))
logData['FindCollectionTarget'] = logData['command'].apply(lambda x: x.get('find', None) if isinstance(x, dict) else "None")
logData['filter'] = logData['command'].apply(lambda x: x.get('filter', None) if isinstance(x, dict) else None)
logData['tokenized_filter'] = logData['filter'].apply(lambda x: tokenize_filter(str(x)) if x is not None else None)
logData['docUUID'] = logData['command'].apply(lambda x: x.get('lsid', {}).get('id', {}).get('$uuid', None) if isinstance(x, dict) else None)
logData['db'] = logData['command'].apply(lambda x: x.get('$db', None) if isinstance(x, dict) else None)
logData['db'] = logData['command'].apply(lambda x: x.get('$db', None) if isinstance(x, dict) else None)
logData['filter_str'] = logData['filter'].apply(lambda x: str(x) if x is not None else None)
logData['returnDataSize'] = logData['command'].apply(lambda x: x.get('nreturned', None) if isinstance(x, dict) else None)
#nreturned
logData['cpuNanos'].head()

0    176371
1    112781
2     71093
3     81173
4    141175
Name: cpuNanos, dtype: int64

In [5]:
labelData = pd.read_json("../No-SQL_Gen/No-SqlDataset.json")
labelData['text'] = labelData['text'].apply(lambda x: x.replace('"', "'") if isinstance(x, str) else x)
labelData.head()
merged_df = pd.merge(logData[['t','filter_str','remote','FindCollectionTarget','db','docUUID', 'nreturned', 'cpuNanos', 'planningTimeMicros']], labelData[['text', 'label']], how='left', left_on='filter_str', right_on='text')
merged_df = merged_df.drop('text',axis=1)
merged_df['FindCollectionTargetdeNamed'] = merged_df['FindCollectionTarget'].apply(replace_alpha_words)
merged_df['filter_str'] = merged_df['filter_str'].apply(lambda x: x.replace('"', "'") if isinstance(x, str) else x)
merged_df['filter_str'] = merged_df['filter_str'].apply(lambda x : str(x))
merged_df['queryLength'] = merged_df['filter_str'].apply(lambda x: len(str(x)))
merged_df['whereDetected'] = merged_df['filter_str'].apply(lambda x: 1 if bool(re.search(r"\$where", x)) else 0)
merged_df['regexDetected'] = merged_df['filter_str'].apply(lambda x: 1 if bool(re.search(r"\$(regex)|\$(elemMatch)", x)) else 0)
merged_df['emptyFilter'] = merged_df['filter_str'].apply(lambda x: 1 if bool(re.search(r"^\{\}$", x)) else 0)
merged_df['nullArgument'] = merged_df['filter_str'].apply(lambda x: 1 if bool(re.search(r"\{\}'", x)) else 0)
merged_df['logicOperator'] = merged_df['filter_str'].apply(lambda x: 1 if bool(re.search(r"\$gt|\$eq|\$in|\$nin|\$ne|[>=<]{2}", x)) else 0)
merged_df['matcherOption'] = merged_df['filter_str'].apply(lambda x: 1 if bool(re.search(r"\$option", x)) else 0)
merged_df['MathOperator'] = merged_df['filter_str'].apply(lambda x: 1 if bool(re.search(r"\$size", x)) else 0)
merged_df['Size'] = merged_df['filter_str'].apply(lambda x: 1 if bool(re.search(r"\$size|this|length", x)) else 0)
merged_df['thisPointer'] = merged_df['filter_str'].apply(lambda x: 1 if bool(re.search(r"this", x)) else 0)
merged_df['denamed'] = merged_df['filter_str'].apply(replace_alpha_words)
merged_df['denamed'] = merged_df['denamed'].apply(replace_this_var)
merged_df['denamed'] = merged_df['denamed'].apply(replace_nums)
merged_df['denamedDb'] = merged_df['db'].apply(replace_alpha_words)
merged_df['ipHash'] = merged_df['remote'].apply(lambda ip: hash(int(ipaddress.IPv4Address(ip.split(':')[0]))))
merged_df['port'] = merged_df['remote'].apply(lambda ip: ip.split(':')[1])
merged_df = merged_df.dropna()


In [6]:
final_df = merged_df[['filter_str','denamed','queryLength','whereDetected','regexDetected','logicOperator','MathOperator','Size','thisPointer','matcherOption','cpuNanos','planningTimeMicros','label']]
print(final_df.head())
print("Shape of the DataFrame:", final_df.shape)
print("\nData Types:")
print(final_df.dtypes)
print("\nDescriptive Statistics:")
print(final_df.describe())
print("\nNull Values:")
print(final_df.isnull().sum())

final_df.to_csv('../Dataset/final.csv', index=False)

stats = {
    'shape': final_df.shape,
    'dtypes': final_df.dtypes.apply(lambda x: x.name).to_dict(),
    'descriptive_stats': final_df.describe().to_dict(),
    'null_values': final_df.isnull().sum().to_dict()
}

with open('../Dataset/metadata.json', 'w') as file:
    json.dump(stats, file, indent=4)

                         filter_str               denamed  queryLength  \
0  {'user': '{}', 'password': '{}'}  {'': '{}', '': '{}'}           32   
1  {'user': '{}', 'password': '{}'}  {'': '{}', '': '{}'}           32   
2   {'username': {'$regex': '^{}'}}  {'': {'$regex': ''}}           31   
3   {'username': {'$regex': '^{}'}}  {'': {'$regex': ''}}           31   
4       {'username': {'$ne': '{}'}}   {'': {'$ne': '{}'}}           27   

   whereDetected  regexDetected  logicOperator  MathOperator  Size  \
0              0              0              0             0     0   
1              0              0              0             0     0   
2              0              1              0             0     0   
3              0              1              0             0     0   
4              0              0              1             0     0   

   thisPointer  matcherOption  cpuNanos  planningTimeMicros  label  
0            0              0    176371                83.0    0.